<a href="https://colab.research.google.com/github/monikabuss/ISYS5002-2024-semester2/blob/main/09_sqlite3_notebook(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

something write read or delete

#SQL and Python (and a little Pandas)

When it comes to data retrieval and basic statistics, SQL shines, while Python shines for in-depth, flexible exploratory data analysis or data science. What if you could combine the two programming languages into a single application?

We will use SQLite3 which is part of the standard Python 3 package, so nothing to install. Let’s take a quick look at the data types that are available:

| SQL | Python |
|-----|--------|
| NULL — Includes a NULL value | none
| INTEGER — Includes an integer | int
| REAL — Includes a floating-point (decimal) value | float
| TEXT. — Includes text | str
| BLOB. — Includes a binary large object that is stored exactly as input | bytes

In this notebook, we’ll demonstrate

* Loading the library
* Creating and connecting to your database
* Creating database tables
* Adding data
* Querying data
* Deleting data



## Import the library

Let’s start off the tutorial by loading in the library. We can do this by using the following command

In [1]:
import sqlite3

## Create a connection

We use the connect method and pass the name of the database. Let's create an *superheroes.db*.

In [2]:
conn = sqlite3.connect('superheroes.db')

We’ve created a new connection object, as well as a new file called *superheroes.db* in the directory in which you’re working.


## Create a cursor object

Now that we’ve created a database connection object, our next task is to create a cursor object. A cursor object allows us to execute SQL queries against a database. A cursor acts a middleware between a connection and SQL query

In [3]:
cur = conn.cursor() #something like 'row' returned from a table

# Create Table

Use a common 'pattern'

1. Create a SQL command as a string
2. User cursor to execture the command
3. If needed fetch the results

In [4]:
#different attributes in the columns
sql = """
CREATE TABLE IF NOT EXISTS users(
   userid INT PRIMARY KEY,
   fname TEXT,
   lname TEXT,
   superhero TEXT,
   gender TEXT);
"""
#primary key - same idea as keys in dictionaries, primary key is unique (number)
#can use also more than one attribute to use as a key, for example rooms: only room number not efficient but also building number needed

# Execute query and commit transaction

cur.execute(sql)
conn.commit() # send it to database

The IF NOT EXISTS will help us when reconnecting to the database. The query will allow us to check if the table exists, and if it does, nothing is changed.

# Insert into Table

Let’s take a look at how to add data with SQLite in Python to the database we just created. Similar to the table generation query, the query to add data uses the cursor object to execute the query.

In [5]:
'''
INSERT INTO users(userid, fname, lname, superhero, gender)
   VALUES('00001', 'Bruce', 'Wayne', 'Batman', 'male');
'''

"\nINSERT INTO users(userid, fname, lname, superhero, gender)\n   VALUES('00001', 'Bruce', 'Wayne', 'Batman', 'male');\n"

In [6]:
sql = '''
INSERT INTO users(userid, fname, lname, superhero, gender)
   VALUES('00006', 'Bruce', 'Wayne', 'Batman', 'male');
'''

# Execute query and commit transaction
cur.execute(sql)
conn.commit()

In [7]:
sql = '''
SELECT * FROM users;
'''
#show us everything from users
cur.execute(sql)
all_results = cur.fetchall()
(all_results)

[(6, 'Bruce', 'Wayne', 'Batman', 'male')]

Often, when we’re working within Python, we’ll have variables that hold values for us. For example, we may have a tuple that contains that information about a user which might look like this:

In [8]:
user = ('00002', 'Sue', 'Storm', 'Invisible Woman', 'female') # MUST BE A TUPLE!

If we wanted to load this data into our database, we would use a different convention:



In [9]:
cur.execute("INSERT INTO users VALUES(?, ?, ?, ?, ?);", user)
conn.commit()

Incidentally, using the (?, ?, …) method we noted above also helps protect against SQL injection attacks.

It’s important to note here that the **SQLite expects the values to be in tuple-format**. However, the variable can contain a list, as long as the list items are tuples. For example, we could add more users using the variable:

In [10]:
more_users = [('00003', 'Peter', 'Parker', 'Spider Man', 'male'),
              ('00004', 'Tony', 'Stark', 'Ironman', 'male'),
              ('00005', 'Diana', 'Princess', 'Wonder Woman', 'female')]

In [11]:
cur.executemany("INSERT INTO users VALUES(?, ?, ?, ?, ?);", more_users)
conn.commit()

In this case, instead of using the execute function, we’ll want to use the executemany function

## Fetching Data

Here we take a look at how to select data with SQLite in Python! We’ll follow a similar structure as we did to execute queries above

In [12]:
sql = '''
SELECT * FROM users;
'''
rows = cur.execute(sql)
for row in rows:
  print(row)

(6, 'Bruce', 'Wayne', 'Batman', 'male')
(2, 'Sue', 'Storm', 'Invisible Woman', 'female')
(3, 'Peter', 'Parker', 'Spider Man', 'male')
(4, 'Tony', 'Stark', 'Ironman', 'male')
(5, 'Diana', 'Princess', 'Wonder Woman', 'female')


Some useful methods

In [13]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
one_result = cur.fetchone()
print(one_result)

(6, 'Bruce', 'Wayne', 'Batman', 'male')


We wanted to return more than only one result, we could use the fetchmany()

In [14]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
three_results = cur.fetchmany(3)
print(three_results)

[(6, 'Bruce', 'Wayne', 'Batman', 'male'), (2, 'Sue', 'Storm', 'Invisible Woman', 'female'), (3, 'Peter', 'Parker', 'Spider Man', 'male')]


We canuse the fetchall() function to return all the results.

In [15]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
print(all_results)

[(6, 'Bruce', 'Wayne', 'Batman', 'male'), (2, 'Sue', 'Storm', 'Invisible Woman', 'female'), (3, 'Peter', 'Parker', 'Spider Man', 'male'), (4, 'Tony', 'Stark', 'Ironman', 'male'), (5, 'Diana', 'Princess', 'Wonder Woman', 'female')]


In [16]:
sql = '''
SELECT superhero, gender FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
print(all_results)

[('Batman', 'male'), ('Invisible Woman', 'female'), ('Spider Man', 'male'), ('Ironman', 'male'), ('Wonder Woman', 'female')]


## Delete Data

In [17]:
sql='''
DELETE FROM users WHERE userid='6';
'''
cur.execute(sql)
conn.commit()

In [18]:
sql='''
DELETE FROM users WHERE lname='Parker';
'''
cur.execute(sql)
conn.commit()

Did it work?

In [19]:
sql='''
select * from users where lname='Parker';
'''
cur.execute(sql)
cur.fetchall()

[]

In [20]:
sql='''
select * from users where gender='female';
'''
cur.execute(sql)
cur.fetchall()

[(2, 'Sue', 'Storm', 'Invisible Woman', 'female'),
 (5, 'Diana', 'Princess', 'Wonder Woman', 'female')]

This prints out an empty list, confirming that the record has been deleted.

## pandas

We can use the pandas package to read a SQLite database

In [21]:
import pandas as pd

sql='''
SELECT * from users;
'''
# Load the data into a DataFrame
users_df = pd.read_sql_query(sql, conn)
users_df

,userid,fname,lname,superhero,gender
0,2,Sue,Storm,Invisible Woman,female
1,4,Tony,Stark,Ironman,male
2,5,Diana,Princess,Wonder Woman,female


and create table from a dataframe

In [25]:
# Select only data for females
users_female_df = users_df[users_df.gender == 'female']

# Write the new DataFrame to a new SQLite table
users_female_df.to_sql("females", conn, if_exists='replace')

2

In [26]:
sql='''
SELECT * from females;
'''
# Load the data into a DataFrame
users_df = pd.read_sql_query(sql, conn)
users_df

,index,userid,fname,lname,superhero,gender
0,0,2,Sue,Storm,Invisible Woman,female
1,2,5,Diana,Princess,Wonder Woman,female


## Activity

- insert Spiderman info
- create 'male' sperheroes table

In [41]:
sql = '''
INSERT INTO users(userid, fname, lname, superhero, gender)
   VALUES('00001', 'Flash', 'Flash', 'Flash', 'male');
'''

# Execute query and commit transaction
cur.execute(sql)
conn.commit()





In [35]:
sql = '''
SELECT * FROM users;
'''
users_df = pd.read_sql_query(sql, conn)


# Select only data for males
users_male_df = users_df[users_df.gender == 'male']

# Write the new DataFrame to a new SQLite table
users_male_df.to_sql("males", conn, if_exists='replace')

2

## View Schema

Was the new table added.  Lets have a look at the schema.

In [36]:
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cur.execute(sql)
cur.fetchall()

[('users',), ('females',), ('males',)]

Now lets get details on a table

In [37]:
sql='''
PRAGMA table_info('users');
'''
cur.execute(sql)
cur.fetchall()
#last one 0: not a ey, 1 is a key

[(0, 'userid', 'INT', 0, None, 1),
 (1, 'fname', 'TEXT', 0, None, 0),
 (2, 'lname', 'TEXT', 0, None, 0),
 (3, 'superhero', 'TEXT', 0, None, 0),
 (4, 'gender', 'TEXT', 0, None, 0)]

## Delete (Drop) Table

When the IF EXISTS clause is used along with DROP TABLE syntax SQLite will not report any error message if the table does not exist.

In [38]:
sql='''
DROP TABLE IF EXISTS males;
'''
cur.execute(sql)
cur.fetchall()

[]

Did it work?

In [39]:
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cur.execute(sql)
cur.fetchall()

[('users',), ('females',)]

## Activity - Order the table by 'userid' and display the result

In [44]:
#first call all users
sql = '''
SELECT * FROM users ORDER BY userid;
'''
cur.execute(sql)
all_results=cur.fetchall
all_results

<function Cursor.fetchall()>

## Close the connection

The connection must be closed at the end of the session because otherwise someone else can edit or we accidentally damage it

In [45]:
conn.close()